<a href="https://colab.research.google.com/github/AstroBoy1/santander/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
***

In [1]:
!pip3 uninstall xgboost
!pip3 install xgboost
!pip uninstall sklearn
!pip install sklearn
!pip install scikit-optimize
!pip install vecstack
!pip install catboost

Uninstalling xgboost-0.7.post4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/xgboost-0.7.post4-py3.6.egg-info
    /usr/local/lib/python3.6/dist-packages/xgboost/*
Proceed (y/n)? y
  Successfully uninstalled xgboost-0.7.post4
    100% |████████████████████████████████| 114.0MB 336kB/s 
Uninstalling sklearn-0.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/sklearn-0.0-py3.6.egg-info
Proceed (y/n)? y
  Successfully uninstalled sklearn-0.0
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
    100% |████████████████████████████████| 81kB 3.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/6d/ca/bce17942bcf7c267b13c97c9c95e2f0ecf0b42160e6074f448
Successfully built vecstack
    100% |████████████████████████████████| 60.1MB 687kB/s 


In [2]:
from google.colab import drive
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import svm
from collections import Counter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from vecstack import stacking
from vecstack import StackingTransformer
import pandas as pd
from glob import glob
import re 
np.random.seed(0) # ensure reproducibility
np.set_printoptions(suppress = True)
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss
# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras.wrappers.scikit_learn import KerasClassifier
# Stacking
from vecstack import stacking
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras import callbacks
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
#from skopt import BayesSearchCV

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train_fn = '/content/gdrive/My Drive/santander_data/train.csv'
valid_fn = '/content/gdrive/My Drive/santander_data/test.csv'
pred_fn = '/content/gdrive/My Drive/santander_data/submission12.csv'
train_data_df = pd.read_csv(train_fn)
test_data_df = pd.read_csv(valid_fn)
train_data_x = train_data_df.drop(columns=["ID_code", "target"]).values
train_data_y = train_data_df["target"].values
test_data_x = test_data_df.drop(columns=["ID_code"]).values

In [0]:
import warnings
warnings.simplefilter("ignore")

# AUC metric

In [0]:
def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score

# Prepare data

In [16]:
# n_classes = 3
n_classes = 2
# Create data: 500 example, 5 feature, 3 classes
# X, y = make_classification(n_samples=500, n_features=5, 
#                            n_informative=3, n_redundant=1, 
#                            n_classes=n_classes, flip_y=0, 
#                            random_state=0)
length = 1000
X, y = train_data_x[:length], train_data_y[:length]

# Make train/test split
# As usual in machine learning task we have X_train, y_train, and X_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Train shape:', X_train.shape)
print('Test shape: ', X_test.shape)

save_directory = '/content/gdrive/My Drive/santander_results/'

Train shape: (160000, 200)
Test shape:  (40000, 200)


# LGB, XGB, Catboost params

In [0]:
lgb_params = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1}

xgb_params = {'tree_method': 'hist',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1,
}

catboost_params = {'subsample':0.36, #rawdata 0.5  ×2 0.45 ×3 0.36
                            #'custom_loss':'Logloss',
                            'loss_function':'Logloss',
                           'random_strength':0,
                           'max_depth':3,
                           'eval_metric':"AUC",
                           'learning_rate':0.02,
                           #'iterations':60000,
                           'iterations':100,
                           #class_weights=[1,2],
                           'bootstrap_type':'Bernoulli',
                           #rsm=0.045,
                            'l2_leaf_reg':0.3,
                           #'task_type':"GPU",
                           'random_seed':432013,
                           'od_type':"Iter",
                           'border_count':128,
                           'logging_level':'Silent'
                           #has_time= True 
                  }

# NN

In [0]:
# LOGGER
class Logger(callbacks.Callback):
    def __init__(self, out_path='./', patience=10, lr_patience=3, out_fn='', log_fn=''):
        self.auc = 0
        self.path = out_path
        self.fn = out_fn
        self.patience = patience
        self.lr_patience = lr_patience
        self.no_improve = 0
        self.no_improve_lr = 0

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        cv_pred = self.model.predict(self.validation_data[0], batch_size=1024)
        cv_true = self.validation_data[1]
        auc_val = roc_auc_score(cv_true, cv_pred)
        if self.auc < auc_val:
            self.no_improve = 0
            self.no_improve_lr = 0
            print("Epoch %s - best AUC: %s" % (epoch, round(auc_val, 4)))
            self.auc = auc_val
            self.model.save(self.path + self.fn, overwrite=True)
        else:
            self.no_improve += 1
            self.no_improve_lr += 1
            print("Epoch %s - current AUC: %s" % (epoch, round(auc_val, 4)))
            if self.no_improve >= self.patience:
                self.model.stop_training = True
            if self.no_improve_lr >= self.lr_patience:
                lr = float(K.get_value(self.model.optimizer.lr))
                K.set_value(self.model.optimizer.lr, 0.75*lr)
                print("Setting lr to {}".format(0.75*lr))
                self.no_improve_lr = 0

        return

# MODEL DEF
def dnn():
#     inp = Input(shape=(200, 1))
#     d1 = Dense(16, activation='relu')(inp)
#     fl = Flatten()(d1)
#     preds = Dense(1, activation='sigmoid')(fl)
#     model = Model(inputs=inp, outputs=preds)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

logger = Logger(patience=10, out_path=save_directory, out_fn='cv_{}.h5')
#nn_params = {'nb_epoch':32, 'batch_size':256, 'callbacks':[logger], 'verbose':1}
nn_params = {'epochs':32, 'batch_size':256, 'verbose':1}

# Initialize 1st level models

In [0]:
def build_keras_model_1():
    model = Sequential()
    model.add(Dense(64, 
                    input_dim=X_train.shape[1], 
                    kernel_initializer='normal', 
                    activation='relu'))
    model.add(Dense(n_classes, 
                    kernel_initializer='normal', 
                    activation='softmax'))
    model.compile(optimizer='rmsprop', 
                  loss='categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
    return model
# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended.
models_1 = [ 
    GaussianNB(),
    
    #LogisticRegression(random_state=0),
    
    ExtraTreesClassifier(random_state=0, n_jobs=-1, 
                         n_estimators=100, max_depth=3),
                         
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=3),
        
    #XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
    #              n_estimators=100, max_depth=3, categories='auto'),
    XGBClassifier(**xgb_params),           
    #LGBMClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
    #               n_estimators=100, max_depth=3)#,
    LGBMClassifier(**lgb_params),
    CatBoostClassifier(**catboost_params),
    #model.fit(X_train, y_train_, validation_data=(X_valid, y_valid))
    #Pipeline([('sc', StandardScaler()), ('clf', KerasClassifier(build_fn=dnn, **nn_params))])
    #Pipeline([('sc', StandardScaler()), ('clf', Ridge(**ridge_params))])
    #KerasClassifier(build_fn=build_keras_model_1, epochs=2, 
    #                batch_size=32, verbose=0)
]

# Perform stacking

In [0]:
S_train_1, S_test_1 = stacking(models_1,                   # list of models
                               X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               mode='oof_pred',            # mode: oof for train set, fit on full 
                                                           #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info
# S_train_1, S_test_1 = stacking(models_1,                   # list of models
#                                X_train, y_train, X_test,   # data
#                                regression=False,           # classification task (if you need 
#                                                            #     regression - set to True)
#                                mode='oof_pred',            # mode: oof for train set, fit on full 
#                                                            #     train and predict test set once
#                                needs_proba=True,           # predict probabilities (if you need 
#                                                            #     class labels - set to False) 
#                                ##save_dir='.',               # save result and log in current dir 
#                                save_dir=save_directory,                                 #     (to disable saving - set to None)
#                                metric=auc,            # metric: callable
#                                n_folds=5,                  # number of folds
#                                stratified=True,            # stratified split for folds
#                                shuffle=True,               # shuffle the data
#                                random_state=0,             # ensure reproducibility
#                                verbose=1)                  # print all info

task:         [classification]
n_classes:    [2]
metric:       [auc]
mode:         [oof_pred]
n_models:     [3]

model  0:     [XGBClassifier]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

    ----
    MEAN:     [0.77202972] + [0.02372778]
    FULL:     [0.77166960]

    Fitting on full train set...

model  1:     [LGBMClassifier]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

    ----
    MEAN:     [0.78647202] + [0.01994472]
    FULL:     [0.78433913]

    Fitting on full train set...

model  2:     [CatBoostClassifier]
0:	total: 93.4ms	remaining: 9.25s
1:	total: 135ms	remaining: 6.63s
2:	total: 172ms	remaining: 5.58s
3:	total: 210ms	remaining: 5.04s
4:	total: 247ms	remaining: 4.7s
5:	total: 289ms	remaining: 4.52s
6:	total: 333ms	remaining: 4.43s
7:	total: 376ms	remaining: 4.32s
8:	total: 416ms	remaining: 4.21s
9:	total: 467ms	remaining: 4.2s
10:	total: 504ms	remaining: 4.08s
11:	total: 553ms	remaining: 4.05s
12:	total: 594ms	remaining: 3.98s
13:	total: 634ms	remaining: 3.89s
14:	total: 676ms	remaining: 3.83s
15:	total: 718ms	remaining: 3.77s
16:	total: 756ms	remaining: 3.69s
17:	total: 795ms	remaining: 3.62s
18:	total: 831ms	remaining: 3.54s
19:	total: 872ms	remaining: 3.49s
20:	total: 908ms	remaining: 3.42s
21:	total: 948ms	remaining: 3.36s
22:	total: 990ms	remaining: 3.31s
23:	total: 1.03s	remaining: 3.26s
24:	total: 1.07s	remaining: 3.2s
25:	total: 1.1

...

0:	total: 51.9ms	remaining: 5.14s
1:	total: 124ms	remaining: 6.09s
2:	total: 180ms	remaining: 5.83s
3:	total: 228ms	remaining: 5.46s
4:	total: 285ms	remaining: 5.41s
5:	total: 339ms	remaining: 5.31s
6:	total: 386ms	remaining: 5.13s
7:	total: 435ms	remaining: 5s
8:	total: 488ms	remaining: 4.94s
9:	total: 541ms	remaining: 4.87s
10:	total: 593ms	remaining: 4.8s
11:	total: 643ms	remaining: 4.71s
12:	total: 696ms	remaining: 4.66s
13:	total: 752ms	remaining: 4.62s
14:	total: 805ms	remaining: 4.56s
15:	total: 853ms	remaining: 4.48s
16:	total: 913ms	remaining: 4.46s
17:	total: 968ms	remaining: 4.41s
18:	total: 1.01s	remaining: 4.32s
19:	total: 1.07s	remaining: 4.27s
20:	total: 1.13s	remaining: 4.26s
21:	total: 1.18s	remaining: 4.2s
22:	total: 1.23s	remaining: 4.13s
23:	total: 1.28s	remaining: 4.06s
24:	total: 1.34s	remaining: 4.02s
25:	total: 1.39s	remaining: 3.95s
26:	total: 1.44s	remaining: 3.88s
27:	total: 1.48s	remaining: 3.81s
28:	total: 1.53s	remaining: 3.76s
29:	total: 1.59s	remaining: 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


0:	total: 52.2ms	remaining: 5.16s
1:	total: 97.3ms	remaining: 4.77s
2:	total: 144ms	remaining: 4.67s
3:	total: 196ms	remaining: 4.71s
4:	total: 247ms	remaining: 4.68s
5:	total: 298ms	remaining: 4.66s
6:	total: 348ms	remaining: 4.62s
7:	total: 402ms	remaining: 4.62s
8:	total: 455ms	remaining: 4.6s
9:	total: 515ms	remaining: 4.64s
10:	total: 571ms	remaining: 4.62s
11:	total: 617ms	remaining: 4.52s
12:	total: 682ms	remaining: 4.57s
13:	total: 736ms	remaining: 4.52s
14:	total: 789ms	remaining: 4.47s
15:	total: 841ms	remaining: 4.41s
16:	total: 887ms	remaining: 4.33s
17:	total: 934ms	remaining: 4.25s
18:	total: 986ms	remaining: 4.21s
19:	total: 1.04s	remaining: 4.16s
20:	total: 1.09s	remaining: 4.11s
21:	total: 1.14s	remaining: 4.04s
22:	total: 1.19s	remaining: 3.97s
23:	total: 1.24s	remaining: 3.92s
24:	total: 1.29s	remaining: 3.86s
25:	total: 1.33s	remaining: 3.8s
26:	total: 1.39s	remaining: 3.75s
27:	total: 1.44s	remaining: 3.71s
28:	total: 1.49s	remaining: 3.66s
29:	total: 1.54s	remaini

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab


0:	total: 56.2ms	remaining: 5.57s
1:	total: 112ms	remaining: 5.5s
2:	total: 168ms	remaining: 5.44s
3:	total: 220ms	remaining: 5.29s
4:	total: 279ms	remaining: 5.29s
5:	total: 332ms	remaining: 5.2s
6:	total: 382ms	remaining: 5.07s
7:	total: 430ms	remaining: 4.95s
8:	total: 490ms	remaining: 4.95s
9:	total: 540ms	remaining: 4.86s
10:	total: 592ms	remaining: 4.79s
11:	total: 640ms	remaining: 4.69s
12:	total: 689ms	remaining: 4.61s
13:	total: 746ms	remaining: 4.58s
14:	total: 800ms	remaining: 4.53s
15:	total: 854ms	remaining: 4.48s
16:	total: 902ms	remaining: 4.4s
17:	total: 969ms	remaining: 4.41s
18:	total: 1.02s	remaining: 4.34s
19:	total: 1.07s	remaining: 4.29s
20:	total: 1.12s	remaining: 4.22s
21:	total: 1.18s	remaining: 4.19s
22:	total: 1.23s	remaining: 4.13s
23:	total: 1.28s	remaining: 4.05s
24:	total: 1.33s	remaining: 3.99s
25:	total: 1.4s	remaining: 3.97s
26:	total: 1.46s	remaining: 3.93s
27:	total: 1.51s	remaining: 3.88s
28:	total: 1.56s	remaining: 3.82s
29:	total: 1.62s	remaining

# Stacking Pipeline

In [1]:
help(StackingTransformer)

NameError: ignored

In [0]:
# Specify steps of Pipeline
pipe_models_1 = [ 
    ('gnb', GaussianNB()),
    #('lg', LogisticRegression(random_state=0)),
    #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0, callbacks=[logger])),
    #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0)),
    #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    ('xgb', XGBClassifier(**xgb_params)),           
    ('lgbm', LGBMClassifier(**lgb_params)),
    ('cb', CatBoostClassifier(**catboost_params))
]
pipe_models_2 = [ 
    #('gnb', GaussianNB()),
    #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    ('xgb', XGBClassifier(**xgb_params)),           
    ('lgbm', LGBMClassifier(**lgb_params)),
    ('cb', CatBoostClassifier(**catboost_params))
]
pipe_models_3 = [ 
    #('gnb', GaussianNB()),
    #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    ('xgb', XGBClassifier(**xgb_params)),           
    ('lgbm', LGBMClassifier(**lgb_params)),
    #('cb', CatBoostClassifier(**catboost_params))
]

stack1 = StackingTransformer(pipe_models_1,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

# stack2 = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
#                       n_estimators=100, max_depth=3)

stack2 = StackingTransformer(pipe_models_2,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

stack3 = StackingTransformer(pipe_models_3,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

In [0]:
# stack1 = stack1.fit(X_train, y_train)
# S_train = stack1.transform(X_train)

# stack2 = stack2.fit(S_train, y_train)
# S_train = stack2.transform(S_train)

# stack3 = stack3.fit(S_train, y_train)
# S_train = stack3.transform(S_train)

task:         [classification]
n_classes:    [2]
metric:       [auc]
variant:      [A]
n_estimators: [7]

estimator  0: [gnb: GaussianNB]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

    MEAN:     [0.88618129] + [0.00408735]

estimator  1: [lg: LogisticRegression]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, ma

KeyboardInterrupt: ignored

In [0]:
stack4 = LogisticRegression(random_state=0)

steps = [('stack1', stack1),
         ('stack2', stack2),
        ('stack3', stack3),
        ('stack4', stack4)]

# Init Pipeline
pipe = Pipeline(steps)

In [43]:
# Fit
pipe = pipe.fit(X_train, y_train)

task:         [classification]
n_classes:    [2]
metric:       [auc]
variant:      [A]
n_estimators: [4]

estimator  0: [gnb: GaussianNB]
    MEAN:     [0.88928160] + [0.00168644]

estimator  1: [xgb: XGBClassifier]
    MEAN:     [0.81261512] + [0.00264756]

estimator  2: [lgbm: LGBMClassifier]
    MEAN:     [0.85238444] + [0.00486554]

estimator  3: [cb: CatBoostClassifier]
    MEAN:     [0.76248479] + [0.00375792]

Train set was detected.
Transforming...

estimator  0: [gnb: GaussianNB]
    DONE

estimator  1: [xgb: XGBClassifier]
    DONE

estimator  2: [lgbm: LGBMClassifier]
    DONE

estimator  3: [cb: CatBoostClassifier]
    DONE

task:         [classification]
n_classes:    [2]
metric:       [auc]
variant:      [A]
n_estimators: [3]

estimator  0: [xgb: XGBClassifier]
    MEAN:     [0.89240256] + [0.00224909]

estimator  1: [lgbm: LGBMClassifier]
    MEAN:     [0.87233721] + [0.00223194]

estimator  2: [cb: CatBoostClassifier]
    MEAN:     [0.89135784] + [0.00230937]

Train set

In [45]:
y_pred = pipe.predict_proba(X_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]

Transforming...

estimator  0: [gnb: GaussianNB]
    DONE

estimator  1: [xgb: XGBClassifier]
    DONE

estimator  2: [lgbm: LGBMClassifier]
    DONE

estimator  3: [cb: CatBoostClassifier]
    DONE

Transforming...

estimator  0: [xgb: XGBClassifier]
    DONE

estimator  1: [lgbm: LGBMClassifier]
    DONE

estimator  2: [cb: CatBoostClassifier]
    DONE

Transforming...

estimator  0: [xgb: XGBClassifier]
    DONE

estimator  1: [lgbm: LGBMClassifier]
    DONE



In [46]:
roc_auc_score(y_test, y_pred_final)


0.8882629811388479

In [0]:
# Save Pipeline
import joblib
_ = joblib.dump(pipe, save_directory + 'pipe_with_stack.pkl')
# Load Pipeline
pipe_loaded = joblib.load(save_directory + 'pipe_with_stack.pkl')

# Predict on Kaggle Test

In [29]:
kaggle_pred = pipe.predict_proba(test_data_x)

Transforming...

estimator  0: [gnb: GaussianNB]
    DONE

estimator  1: [etc: ExtraTreesClassifier]
    DONE

estimator  2: [rfc: RandomForestClassifier]
    DONE

estimator  3: [xgb: XGBClassifier]
    DONE

estimator  4: [lgbm: LGBMClassifier]
    DONE

estimator  5: [cb: CatBoostClassifier]
    DONE

Transforming...

estimator  0: [gnb: GaussianNB]
    DONE

estimator  1: [xgb: XGBClassifier]
    DONE

estimator  2: [lgbm: LGBMClassifier]
    DONE

estimator  3: [cb: CatBoostClassifier]
    DONE

Transforming...

estimator  0: [xgb: XGBClassifier]
    DONE

estimator  1: [lgbm: LGBMClassifier]
    DONE

estimator  2: [cb: CatBoostClassifier]
    DONE



In [0]:
output_df = pd.DataFrame()
output_df["ID_code"] = test_data_df["ID_code"]
pred_final = [elem[1] for elem in kaggle_pred]
output_df["target"] = pred_final
output_df.to_csv(save_directory + "predictions.csv", index=False)

# Look at the result

In [0]:
print('We have %d classes and %d models so in resulting arrays \
we expect to see %d columns.' % (n_classes, len(models_1), n_classes * len(models_1)))
print('S_train_1 shape:', S_train_1.shape)
print('S_test_1 shape: ', S_test_1.shape)

We have 2 classes and 6 models so in resulting arrays we expect to see 12 columns.
S_train_1 shape: (8000, 12)
S_test_1 shape:  (2000, 12)


In [0]:
S_train_1[:2]

array([[0.88595608, 0.11404392, 0.87512032, 0.12487968, 0.90076603,
        0.09923397, 0.91364649, 0.08635351, 0.86732459, 0.13267542,
        0.91284642, 0.08715358],
       [0.99596026, 0.00403974, 0.98994982, 0.01005018, 0.90782715,
        0.09217285, 0.91324549, 0.08675451, 0.96264553, 0.03735447,
        0.96822305, 0.03177695]])

In [0]:
S_test_1[:2]

array([[0.96200187, 0.03799813, 0.94323898, 0.05676102, 0.89576226,
        0.10423774, 0.89789474, 0.10210526, 0.92684668, 0.07315334,
        0.93933053, 0.06066947],
       [0.99292921, 0.00707079, 0.9939683 , 0.0060317 , 0.90598083,
        0.09401917, 0.90470871, 0.09529129, 0.95539749, 0.04460249,
        0.95137477, 0.04862523]])

In [0]:
S_train_1[0][0], S_train_1[0][1], S_train_1[0][1]

(0.8859560775904103, 0.11404392240959342, 0.11404392240959342)

# Our arrays and log were saved in current dir

In [0]:
names = sorted(glob(save_directory + '*.npy'))
npy_1_name = names[0] # for later use

print('Arrays:')
for name in names:
    print(name)

names = sorted(glob(save_directory + '*.log.txt'))
log_1_name = names[0] # for later use

print('\nLogs:')
for name in names:
    print(name)

Arrays:
/content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy

Logs:
/content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.log.txt


# Time to collect results

After several (many) days of building, optimizing, and testing models we have a lot of files with saved OOF.  
At this point we can load and use OOF of specific model or all OOF we have.

# Find specific model

We can open logs and find the model of interest.  
We can do it programmatically or just open logs in editor.  
Name of the `.log.txt` file is the same as the name of corresponding `.npy` file (except extension).  
To find columns containing OOF of specific model we use model index from log:
* if we predicted class labels - corresponding column index is just model index
* if we predicted probabilities - corresponding column index is model index multiplied by number of classes

In [0]:
print("Let's open this log: %s" % log_1_name)
with open(log_1_name) as f:
    lines = f.readlines()

print("Let's look what models did we build in those session.\n")
for line in lines:
    if re.search(r'^model [0-9]+', line):
        print(line)

Let's open this log: [2018.02.01].[15.41.41].305268.0eadc0.log.txt
Let's look what models did we build in those session.

model 0:    [GaussianNB]

model 1:    [LogisticRegression]

model 2:    [ExtraTreesClassifier]

model 3:    [RandomForestClassifier]

model 4:    [XGBClassifier]

model 5:    [LGBMClassifier]

model 6:    [KerasClassifier]



# Load specific model OOF

Let's say we are interested in `LGBMClassifier`.  
We found out that it has index 5.  
Then we load target `.npy` file and because of probabilities we need 3 columns from 15 (5 \* 3) to 18 (5 \* 3 + 3)

In [0]:
print("Let's load this .npy file: %s" % npy_1_name)
S = np.load(npy_1_name)
S_train_lgbm = S[0][:, 15:18]
S_test_lgbm = S[1][:, 15:18]

Let's load this .npy file: /content/gdrive/My Drive/santander_results/[2019.03.26].[04.42.41].477166.c4aa2f.npy


In [0]:
S_train_lgbm[:5]

array([[ 0.00040829,  0.00281319,  0.99677852],
       [ 0.99732125,  0.00258249,  0.00009626],
       [ 0.98322854,  0.01610955,  0.00066191],
       [ 0.00107737,  0.99633895,  0.00258368],
       [ 0.97101719,  0.02843959,  0.00054321]])

In [0]:
S_test_lgbm[:5]

array([[ 0.60639131,  0.3588515 ,  0.03475718],
       [ 0.03609523,  0.90174785,  0.06215692],
       [ 0.08650007,  0.89717473,  0.0163252 ],
       [ 0.00068572,  0.98858075,  0.01073353],
       [ 0.00122693,  0.99814513,  0.00062793]])

# Compute score of specific model

In [0]:
print('LGBMCLassifier log loss: %.8f' % log_loss(y_train, S_train_lgbm))

LGBMCLassifier log loss: 0.41430248


# Load ALL OOF

***Note:*** If you load OOF from scratch, don't forget to load `y_train` from initial dataset too.

In [0]:
# Create empty arrays
S_train_all = np.zeros((X_train.shape[0], 0))
S_test_all = np.zeros((X_test.shape[0], 0))

# Load results
for name in sorted(glob(save_directory + '*.npy')):
    print('Loading: %s' % name)
    S = np.load(name)
    S_train_all = np.c_[S_train_all, S[0]]
    S_test_all = np.c_[S_test_all, S[1]]
    
print('\nS_train_all shape:', S_train_all.shape)
print('S_test_all shape: ', S_test_all.shape)

Loading: /content/gdrive/My Drive/santander_results/[2019.03.27].[21.14.02].232515.7f0aac.npy

S_train_all shape: (8000, 6)
S_test_all shape:  (2000, 6)


# Apply 2nd level model

Just use class 0 probability

In [0]:
columns = [n for n in range(0, S_train_all.shape[1], 2)]
l2_train = S_train_all[:, columns]
l2_test = S_test_all[:, columns]
l2_train.shape, l2_test.shape

((8000, 3), (2000, 3))

In [0]:
# Initialize 2nd level model
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
#model = LogisticRegression(random_state=0)
    

# Fit 2nd level model
# model = model.fit(S_train_all, y_train)
model = model.fit(l2_train, y_train)


# Predict
# y_pred = model.predict_proba(S_test_all)
y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

0.7963757860769392

In [0]:
# Initialize 2nd level model
#model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
#                      n_estimators=100, max_depth=3)
model = LogisticRegression(random_state=0)
    

# Fit 2nd level model
# model = model.fit(S_train_all, y_train)
model = model.fit(l2_train, y_train)


# Predict
# y_pred = model.predict_proba(S_test_all)
y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7624175376380798

No change with redundant features

# redundant features test

In [0]:
# Initialize 2nd level model
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    

# Fit 2nd level model
model = model.fit(S_train_all, y_train)
#model = model.fit(l2_train, y_train)


# Predict
y_pred = model.predict_proba(S_test_all)
#y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

0.8461037566493023

# Predictions on the test set

In [0]:
# Initialize 2nd level model
#model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
#                      n_estimators=100, max_depth=3)
model = LogisticRegression(random_state=0)
    

# Fit 2nd level model
# model = model.fit(S_train_all, y_train)
model = model.fit(l2_train, y_train)


# Predict
# y_pred = model.predict_proba(S_test_all)
y_pred = model.predict_proba(l2_test)

from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
# roc_auc_score(y_test, y_pred_final)